## **IMPORTING THE NECESSARY LIBRARIES**

In [ ]:
# !pip install fastai --upgrade

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from PIL import Image
# import fastai
from fastai.vision.all import *

## **INTERPRETATION OF THE DATA**

In [2]:
train_path = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/'
test_path = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/'
sample_image = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/J/J1.jpg'
img = Image.open(sample_image)
w, h = img.size
print("Width of image:", w)
print("Height of image:", h)
img

In [3]:
db = DataBlock(blocks = (ImageBlock, CategoryBlock), get_items = get_image_files, get_y = parent_label, splitter = RandomSplitter(valid_pct = 0.2), item_tfms = Resize(128))
file = get_image_files(train_path)
print(file[1])
print(file[1].parent.name)

In [4]:
dataloaders = db.dataloaders(train_path, bs = 32)

In [5]:
dataloaders.show_batch()

## **TRAINING THE MODEL**

In [6]:
model_temp = cnn_learner(dataloaders, models.vgg16_bn, metrics = [accuracy], loss_func = CrossEntropyLossFlat())

In [8]:
model_temp.fine_tune(5)

In [9]:
model_temp.show_results()

## **TESTING THE MODEL**

In [10]:
test_images_temp = get_image_files(test_path) 
test_dataloaders_temp = model_temp.dls.test_dl(test_images_temp) 
pred_temp, _ = model_temp.get_preds(dl = test_dataloaders_temp) 
preds_temp = pred_temp.argmax(dim = -1).numpy() 

In [12]:
df_test_temp = pd.DataFrame(os.listdir(test_path)).rename(columns={0:'Image_Name'}) 
df_test_temp['Actual_Value'] = df_test_temp['Image_Name'].str.split('_').str[0]
df_test_temp['Predicted_Value'] = preds_temp
vocab = dict(zip(np.arange(29), dataloaders.vocab))
df_test_temp['Predicted_Value'] = df_test_temp['Predicted_Value'].replace(vocab)
df_test_temp['Image_Path'] = test_path + df_test_temp['Image_Name']
fig, axes = plt.subplots(nrows=4, ncols=7, figsize=(20, 12),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_test_temp['Image_Path'].iloc[i]))
    ax.set_title(f"True: {df_test_temp.Actual_Value.iloc[i]}\nPredicted: {df_test_temp.Predicted_Value.iloc[i]}", fontsize = 15)
plt.tight_layout()
plt.show()